In [ ]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

In [1]:
import os
import json
from dotenv import load_dotenv
from chromadb import Client as ChromaClient
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

ModuleNotFoundError: No module named 'chromadb'

# Load environment variables


In [ ]:
CHROMA_PATH = os.getenv("CHROMA_DB_PATH", "./chroma_db")
LLAMA_MODEL_NAME = os.getenv("LLAMA_MODEL", "meta-llama/Meta-Llama-3-8B")
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL", "sentence-transformers/all-MiniLM-L6-v2")

# Initialize ChromaDB


In [ ]:
chroma_client = ChromaClient(path=CHROMA_PATH)
collection = chroma_client.get_or_create_collection(name="my_collection")

# Set embeddings


In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL_NAME)
Settings.chunk_size = 512
Settings.chunk_overlap = 25

# Load documents and store in ChromaDB


In [ ]:
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)

for doc in documents:
    collection.add(
        ids=[doc.metadata.get("doc_id", str(hash(doc.text)))],
        documents=[doc.text],
        metadatas=[doc.metadata]
    )

# Configure retriever


In [ ]:
retriever = VectorIndexRetriever(index=index, similarity_top_k=3)
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

# Load LLaMA 3 model


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA_MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(LLAMA_MODEL_NAME, device_map="auto")

def query_documents(query):
    response = query_engine.query(query)
    text = response.source_nodes[0].text.replace('\n', ' ').strip()
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=600)
    return tokenizer.batch_decode(outputs)[0]


# Process queries

In [2]:
import csv

file_path = "./queries.json"
with open(file_path, 'r') as f:
    data = json.load(f)

answers = []
for query in data:
    query_id = query["query_id"]
    question = query["question"]
    answer_text = query_documents(question)
    
    answers.append({
        "id": query_id,
        "answer": answer_text
    })

# Write the results to a CSV file
with open('results.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'answer']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for answer in answers:
        writer.writerow(answer)
        # Write the results to a CSV file with the specified format
        with open('results.csv', 'w', newline='') as csvfile:
            fieldnames = ['ID', 'context', 'answer', 'references']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for answer in answers:
                writer.writerow({
                    'ID': answer['id'],
                    'context': answer['context'],
                    'answer': answer['answer'],
                    'references': json.dumps(answer['references'])
                })


NameError: name 'json' is not defined